In [2]:
import yfinance as yf
import pandas as pd

Skidanje cijena pomoću yfinance paketa:

In [3]:
tickers = ["SPY", "AGG", "GC=F", "VNQ"]

prices = yf.download(tickers, start="2012-01-01", end="2023-12-31")["Adj Close"]

prices

[*********************100%***********************]  4 of 4 completed


Ticker,AGG,GC=F,SPY,VNQ
Date,,,,
2012-01-03 00:00:00+00:00,79.201958,1599.699951,101.091988,35.285416
2012-01-04 00:00:00+00:00,79.093979,1611.900024,101.250511,34.682125
2012-01-05 00:00:00+00:00,79.201958,1619.400024,101.520073,35.007904
2012-01-06 00:00:00+00:00,79.266724,1616.099976,101.258453,34.893276
2012-01-09 00:00:00+00:00,79.201958,1607.500000,101.504257,34.772625
...,...,...,...,...
2023-12-22 00:00:00+00:00,95.852509,2057.100098,469.225250,85.181526
2023-12-26 00:00:00+00:00,96.046486,2058.199951,471.206573,85.812798
2023-12-27 00:00:00+00:00,96.657501,2081.899902,472.058533,86.230392


Čitanje i parsiranje csv dokumenta koji sadrži težine svake klase imovine, ispis težina klasa po godinama

In [4]:
csvfile = pd.read_csv("asset_class_marketcaps.csv", skiprows=1)
marketcap_years = [str(year) for year in range(2012, 2024)]
weights_years = [str(year) + ".1" for year in range(2012, 2024)]

marketcap_df = csvfile[["Asset class"] + marketcap_years]

weights_df = csvfile[["Asset class"] + weights_years]
weights_df.columns = ["Asset class"] + marketcap_years

weights_df = weights_df.set_index("Asset class")

weights_df = weights_df.apply(lambda x: x.str.rstrip("%").astype(float) / 100)

weights_df.style.format("{:,.2%}")

,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
Asset class,,,,,,,,,,,,
Equity,35.06%,40.81%,40.62%,40.64%,40.67%,42.67%,40.41%,42.43%,42.44%,46.09%,45.04%,46.72%
Fixed income,48.21%,46.00%,46.19%,46.63%,46.53%,44.80%,46.90%,44.99%,45.52%,42.45%,42.95%,41.62%
Commodity,12.27%,8.30%,7.74%,7.10%,7.33%,7.15%,7.29%,7.33%,7.93%,6.92%,8.04%,7.93%
Real estate,4.47%,4.89%,5.46%,5.63%,5.46%,5.38%,5.41%,5.25%,4.12%,4.54%,3.97%,3.73%


Pretvaranje cijena tickera u postotne povrate (razlika u cijeni iz dana u dan)

In [5]:
change = prices.pct_change(1, fill_method=None).dropna()
change

Ticker,AGG,GC=F,SPY,VNQ
Date,,,,
2012-01-04 00:00:00+00:00,-0.001363,0.007626,0.001568,-0.017097
2012-01-05 00:00:00+00:00,0.001365,0.004653,0.002662,0.009393
2012-01-06 00:00:00+00:00,0.000818,-0.002038,-0.002577,-0.003274
2012-01-09 00:00:00+00:00,-0.000817,-0.005321,0.002427,-0.003458
2012-01-10 00:00:00+00:00,-0.000273,0.014619,0.008671,0.011103
...,...,...,...,...
2023-12-22 00:00:00+00:00,-0.001314,0.008827,0.002010,0.003891
2023-12-26 00:00:00+00:00,0.002024,0.000535,0.004223,0.007411
2023-12-27 00:00:00+00:00,0.006362,0.011515,0.001808,0.004866


Pretvaranje dnevnih povrata u godišnje povrate radi konteksta

In [6]:
yearly_change = change.groupby(change.index.year).apply(lambda x: (1 + x).prod() - 1)
yearly_change.index.name = "Year"

yearly_change.style.format("{:,.2%}")

Ticker,AGG,GC=F,SPY,VNQ
Year,,,,
2012,3.96%,4.69%,14.17%,16.65%
2013,-1.98%,-28.24%,32.31%,2.30%
2014,6.00%,-1.50%,13.46%,30.40%
2015,0.48%,-10.44%,1.23%,2.43%
2016,3.22%,12.03%,13.01%,5.80%
2017,3.55%,13.59%,21.71%,4.90%
2018,0.68%,-0.96%,-2.94%,-4.05%
2019,8.46%,18.87%,31.22%,28.91%
2020,7.48%,24.59%,18.33%,-4.61%


Računanje težinskih godišnjih povrata svakog tickera u indeksu (koliko je koja klasa pridonjelja godišnjem povratu indeksa)

In [7]:
asset_class_to_ticker_map = {
    "Equity": "SPY",
    "Fixed income": "AGG",
    "Commodity": "GC=F",
    "Real estate": "VNQ",
}

weights_df_copy = weights_df.copy()

weights_df_copy.rename(index=asset_class_to_ticker_map, inplace=True)
weights_df_copy = weights_df_copy.transpose()
weights_df_copy.index = weights_df_copy.index.astype("int64")

weighted_returns = weights_df_copy.multiply(yearly_change)

weighted_returns.style.format("{:,.2%}")

,AGG,GC=F,SPY,VNQ
2012,1.91%,0.58%,4.97%,0.74%
2013,-0.91%,-2.34%,13.18%,0.11%
2014,2.77%,-0.12%,5.47%,1.66%
2015,0.22%,-0.74%,0.50%,0.14%
2016,1.50%,0.88%,5.29%,0.32%
2017,1.59%,0.97%,9.26%,0.26%
2018,0.32%,-0.07%,-1.19%,-0.22%
2019,3.80%,1.38%,13.25%,1.52%
2020,3.40%,1.95%,7.78%,-0.19%
2021,-0.75%,-0.24%,13.24%,1.84%


Godišnji povrati indeksa dobiveni sumiranjem stupaca

In [8]:
index_returns = weighted_returns.sum(axis=1).to_frame()

index_returns.index.name = "Year"
index_returns.columns = ["Index Returns"]

index_returns.style.format("{:,.2%}")

,Index Returns
Year,
2012,8.20%
2013,10.04%
2014,9.78%
2015,0.12%
2016,7.99%
2017,12.09%
2018,-1.16%
2019,19.95%
2020,12.94%


Računamo betu svakog tickera prema formuli:

$$
\beta = \frac{\mathrm{Cov}(R_{\mathrm{stock}}, R_{\mathrm{market}})}{\mathrm{Var}(R_{\mathrm{market}})}
$$

gdje su:

- $R_{stock}$: Povrat dionice
- $R_{market}$: Povrat indeksa koji smo stvorili
- $\mathrm{Cov}(R_{stock}, R_{market})$: Kovarijanca između povrata dionice i povrata indeksa
- $\mathrm{Var}(R_{market})$: Varijanca povrata indeksa

In [ ]:
daily_weights_df = change.index.to_frame()

# Pretpostavka da su weightovi svaki dan isti unutar jedne godine
daily_weights_df["Year"] = daily_weights_df["Date"].dt.year
daily_weights_df = daily_weights_df.merge(weights_df_copy, left_on="Year", right_index=True).set_index("Date")

daily_weights_df = daily_weights_df.drop(columns="Year")

daily_index_change = change.multiply(daily_weights_df).sum(axis=1)

In [10]:
kenneth_french_df = pd.read_csv("F-F_Research_Data_Factors_daily.CSV", skiprows=4)
kenneth_french_df = kenneth_french_df[:-3]

kenneth_french_df["Date"] = pd.to_datetime(kenneth_french_df["Date"], format="%Y%m%d")
kenneth_french_df = kenneth_french_df.set_index("Date")

kenneth_french_df

,Mkt-RF,SMB,HML,RF
Date,,,,
1926-07-01,0.10,-0.25,-0.27,0.009
1926-07-02,0.45,-0.33,-0.06,0.009
1926-07-06,0.17,0.30,-0.39,0.009
1926-07-07,0.09,-0.58,0.02,0.009
1926-07-08,0.21,-0.38,0.19,0.009
...,...,...,...,...
2024-09-20,-0.27,-0.72,-0.64,0.020
2024-09-23,0.21,-0.85,-0.08,0.020
2024-09-24,0.24,0.12,-0.58,0.020


In [54]:
change.index = change.index.tz_localize(None)
daily_index_change.index = daily_index_change.index.tz_localize(None)

start_date = change.index[0]
end_date = change.index[-1]

filtered_kenneth_french_df = kenneth_french_df.loc[start_date:end_date]["RF"][1:] / 100
filtered_kenneth_french_df = filtered_kenneth_french_df.reindex(change.index, fill_value=0)

In [55]:
excess_index = daily_index_change.add(-filtered_kenneth_french_df, axis="index")
excess_change = change.add(-filtered_kenneth_french_df, axis="index")

In [ ]:
def linear_regression(stock_returns=excess_change, index_returns=excess_index):
    """
    Implement linear regression for CAPM from scratch using ordinary least squares (OLS).
    
    Parameters:
    -----------
    stock_returns: Series of excess stock returns
    market_returns: Series of excess market returns

    Returns:
    --------
    alpha: Intercept term (risk-adjusted excess return)
    beta: Slope (systematic risk)
    r_squared: R-squared value (goodness of fit)
    """

    result = pd.DataFrame(columns=["Ticker", "Alpha", "Beta", "R_squared"])

    n = len(excess_change)

    x_mean = excess_index.mean()
    
    for stock in stock_returns.columns:
        selected_stock = stock_returns[stock]
        y_mean = selected_stock.mean()
        
        xy_sum = sum((index_returns - x_mean) * (selected_stock - y_mean))

        x_square_sum = sum((index_returns - x_mean)**2)

        beta = xy_sum / x_square_sum
        alpha = y_mean - beta * x_mean

        # R-squared
        y_pred = alpha + beta * index_returns
        ss_res = selected_stock.sub(y_pred).pow(2).sum() # residual sum of squares
        ss_tot = selected_stock.sub(y_mean).pow(2).sum()  # total sum of squares
        r_squared = 1 - (ss_res / ss_tot)

        row = pd.DataFrame({
            "Ticker": [stock],
            "Alpha": [alpha],
            "Beta": [beta],
            "R_squared": [r_squared]
        })
        result = pd.concat([result, row], ignore_index=True)

    result.set_index("Ticker", inplace=True)

    return result

result = linear_regression()
result

,Alpha,Beta,R_squared
Ticker,,,
AGG,-0.000022,0.210798,0.137067
GC=F,-0.000017,0.467243,0.065283
SPY,0.000041,1.843457,0.871428
VNQ,-0.000167,1.855788,0.642026


Račun bete na godišnjoj razini povrata dionice i indeksa:

In [12]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
def yearly_betas(change= change, daily_index_change = daily_index_change, tickers = tickers, start_year = 2012, end_year= 2024):
    yearly_betas = pd.DataFrame(columns=['Year', 'Ticker', 'Beta'])
    
    temp1 = change.copy()
    temp2 = daily_index_change.copy()
    
    for ticker in tickers:
        for year in range(start_year, end_year + 1):
            ticker_changes = temp1[ticker].loc[temp1.index.year == year]
            index_changes = temp2.loc[temp2.index.year == year]
            beta = ticker_changes.cov(index_changes) / index_changes.var()
            yearly_betas = pd.concat(
                [yearly_betas, pd.DataFrame({'Year': [year], 'Ticker': [f"β ({ticker})"], 'Beta': [beta]})],
                ignore_index=True
            )
    
    yearly_betas = yearly_betas.pivot(index="Year", columns="Ticker", values="Beta")
    
    return yearly_betas[:-1]


yearly_betas = yearly_betas()

yearly_betas

/tmp/ipykernel_2119/3112607467.py:16: RuntimeWarning: invalid value encountered in scalar divide
  r_squared = 1 - (ss_res / ss_tot)


,Year,Ticker,Beta,R_squared
0,2012,β (SPY),2.120995,0.812326
1,2013,β (SPY),1.560215,0.765672
2,2014,β (SPY),2.195741,0.825904
3,2015,β (SPY),2.062416,0.858647
4,2016,β (SPY),2.033425,0.814086
5,2017,β (SPY),1.696735,0.637329
6,2018,β (SPY),2.228643,0.938896
7,2019,β (SPY),2.128359,0.829511
8,2020,β (SPY),1.874215,0.928746
9,2021,β (SPY),1.791428,0.903523


##### Backtesting po povijesnim podacima koristeći CAPM, prozor od 3 godine.

In [14]:
Rf_rate = yf.download("^IRX", start="2012-01-01", end="2023-12-31")["Adj Close"].dropna()
Rf_rate

[*********************100%***********************]  1 of 1 completed


Ticker,^IRX
Date,
2012-01-03 00:00:00+00:00,0.005
2012-01-04 00:00:00+00:00,0.010
2012-01-05 00:00:00+00:00,0.010
2012-01-06 00:00:00+00:00,0.015
2012-01-09 00:00:00+00:00,0.005
...,...
2023-12-22 00:00:00+00:00,5.208
2023-12-26 00:00:00+00:00,5.203
2023-12-27 00:00:00+00:00,5.235


In [15]:
def rolling_beta(start_year, end_year, window, change=change, daily_index_change=daily_index_change, tickers=tickers):
    window_betas = pd.DataFrame(columns=['Window', 'Ticker', 'Beta'])
    
    temp1 = change.copy()
    temp2 = daily_index_change.copy()
    
    for ticker in tickers:
        window_years = range(start_year, end_year)    
        ticker_changes = temp1[ticker][temp1.index.year.isin(window_years)]
        index_changes = temp2[temp2.index.year.isin(window_years)]
        
        beta = ticker_changes.cov(index_changes) / index_changes.var()
    
        window_betas = pd.concat(
            [window_betas, pd.DataFrame({'Window': [end_year], 'Ticker': [ticker], 'Beta': [beta]})],
            ignore_index=True
        )
    
    window_betas = window_betas.pivot(index="Window", columns="Ticker", values="Beta")
    return window_betas

In [16]:
window = 3

CAPM_expected_return = pd.DataFrame(columns=["Year", "Ticker", "Expected Return", "Real Return"])

for i in range(2014, 2024, window):
    start_year = i - window + 1
    end_year = i + 1
    window_changes = yearly_change.loc[end_year] if end_year < 2024 else None # promjena tickera na vremenskom prozoru
    window_beta = rolling_beta(start_year=start_year, end_year=end_year,window=window) # beta izracunana na vremenskom prozoru
    window_rf_rate = Rf_rate.loc[Rf_rate.index.year == end_year-1].iloc[-1].squeeze() / 100 # zadnja vrijednost godišnjeg povrata trezoraca u godini prije one koju predvidamo
    window_index_return = index_returns.loc[start_year:end_year-1].mean().squeeze() # prosjecni godisnji povrat indeksa u prozoru

    for ticker in tickers:
        beta = window_beta[ticker]

        expected_return = window_rf_rate + beta * (window_index_return - window_rf_rate)
        
        if window_changes is not None:
            CAPM_expected_return = pd.concat([
                CAPM_expected_return,
                pd.DataFrame({
                    "Year": [end_year],
                    "Ticker": [ticker],
                    "Expected Return": [expected_return.squeeze()],
                    "Real Return" : [window_changes[ticker]]
                })
            ], ignore_index=True)
        else:
            CAPM_expected_return = pd.concat([
                CAPM_expected_return,
                pd.DataFrame({
                    "Year": [end_year],
                    "Ticker": [ticker],
                    "Expected Return": [expected_return.squeeze()],
                    "Real Return" : [None]
                })
            ], ignore_index=True)

CAPM_expected_return.set_index("Year")

,Ticker,Expected Return,Real Return
Year,,,
2015,SPY,0.176627,0.012343
2015,AGG,0.008659,0.004807
2015,GC=F,0.130154,-0.104401
2015,VNQ,0.168474,0.024325
2018,SPY,0.121689,-0.029365
2018,AGG,0.018833,0.006752
2018,GC=F,0.028747,-0.009563
2018,VNQ,0.123796,-0.040523
2021,SPY,0.204993,0.287288


##### Backtesting povijesnih podataka koristeći isti CAPM model, DNEVNE frekvencije i Kenneth French dnevne bezrizične stope

In [17]:
window = 3

CAPM_expected_return_daily = pd.DataFrame(columns=["Year", "Ticker", "Expected Return", "Real Return"])

for i in range(2014, 2024, window):
    start_year = i - window + 1
    end_year = i + 1
    window_changes_daily = change.loc[str(end_year)].mean() if end_year < 2024 else None
    window_beta_daily = rolling_beta(start_year=start_year, end_year=end_year,window=window) # beta izracunana na vremenskom prozoru
    window_rf_rate_daily = kenneth_french_df["RF"].loc[f"{start_year}-01-01":f"{end_year-1}-12-31"].mean().squeeze() # prosjecna bezrizicna stopa
    window_index_return_daily = daily_index_change.loc[f"{start_year}-01-01":f"{end_year-1}-12-31"].mean().squeeze() # prosjecni dnevni povrat indeksa u prozoru

    for ticker in tickers:
        beta = window_beta_daily[ticker]

        expected_return = window_rf_rate_daily + beta * (window_index_return_daily - window_rf_rate_daily)

        annualized_expected_return = (1 + expected_return.squeeze())**252 - 1
        
        if window_changes_daily is not None:
            annualized_real_return = (1 + window_changes_daily[ticker].mean())**252 - 1
            CAPM_expected_return_daily = pd.concat([
                CAPM_expected_return_daily,
                pd.DataFrame({
                    "Year": [end_year],
                    "Ticker": [ticker],
                    "Expected Return": [annualized_expected_return],
                    "Real Return" : [annualized_real_return]
                })
            ], ignore_index=True)
        else:
            CAPM_expected_return_daily = pd.concat([
                CAPM_expected_return_daily,
                pd.DataFrame({
                    "Year": [end_year],
                    "Ticker": [ticker],
                    "Expected Return": [annualized_expected_return],
                    "Real Return" : [None]
                })
            ], ignore_index=True)

CAPM_expected_return_daily.set_index("Year", inplace=True)
CAPM_expected_return_daily

,Ticker,Expected Return,Real Return
Year,,,
2015,SPY,0.173721,0.024439
2015,AGG,0.014002,0.005529
2015,GC=F,0.127177,-0.094667
2015,VNQ,0.165420,0.039375
2018,SPY,-0.189513,-0.015496
2018,AGG,0.370620,0.007166
2018,GC=F,0.303002,-0.004236
2018,VNQ,-0.198199,-0.028603
2021,SPY,-0.683025,0.298147
